In [2]:
import json
import warnings
warnings.filterwarnings('ignore')

def ecrire_json(chemin, contenu):
  w = open(chemin, "w", encoding="utf-8")
  w.write(json.dumps(contenu, indent=2, ensure_ascii=False))
  w.close()

def ouvrir_json(chemin):
  f = open(chemin, encoding="utf-8")
  toto = json.load(f)
  f.close()
  return toto 

def lire_fichier(chemin):
    with open(chemin,encoding="utf-8",errors="ignore") as f:
        text = f.read()
    return text

#dico forme chemin_article:institution_presente
dico_institutions = {}
#liste établie à partir des 50 premières universités du top Shanghai 2021 (avec variations orthographiques/linguistiques) + GAFAM
liste_institutions = ouvrir_json("liste_institutions.json")

liste_articles = ouvrir_json("liste_chemins_acl_linux.json")

for chemin in liste_articles:
    if ".body" not in chemin:
        article = lire_fichier(chemin)
        chemin_nettoye = chemin.split("/")[-1].split("\\")[-1]

        #partie dico_institutions
        for insti in liste_institutions:
            if insti in article[:1000]:
                if chemin_nettoye not in dico_institutions:
                    dico_institutions[chemin_nettoye]=[]
                dico_institutions[chemin_nettoye].append(insti)
        
ecrire_json("dico_institutions1000_corr.json",dico_institutions)

In [3]:
print(len(dico_institutions)) #6459 sur tout le texte, 3094 sur 500 premiers caractères, 3314 sur 1000 premiers
print(0)
print(len(liste_articles))

2603
0
8719


In [16]:
#comment croiser les données lol (relier les articles qui contiennent des noms d'institutions au dico avec les strengths)
dico_strengths = ouvrir_json("stopwords/comparaison_claims_chiffres_equilibre_sw_corr.json")
dico_i = ouvrir_json("dico_institutions1000_corr.json")
dico_institutions = ouvrir_json("dico_institutions1000_corr.json")

for chemin in dico_institutions.keys():
    if chemin in dico_strengths:
        #if ".cite" not in chemin:
            dico_institutions[chemin].append(dico_strengths[chemin])
        
#print(dico_strengths)
print(len(dico_institutions))
ecrire_json("comparaison_claims_chiffres22_institutions_sw_corr.json",dico_institutions)

0


In [8]:
#faire dico inverse avec seulement les articles qui contiennent un nom d'institution pour relancer calculs seulement sur ces dicos

#dico sans les noms des insti, seulement une version diminuée comparaison_claims_chiffres_equilibre2 avec slmt valeurs des articles qui contiennent un nom d'insti
dico_slmt_insti = {}
for chemin in dico_strengths.keys():
    if chemin in dico_institutions and ".cite" not in chemin:
        dico_slmt_insti[chemin]=dico_strengths[chemin]
        
print(len(dico_slmt_insti))
#print(dico_slmt_insti)
ecrire_json("comparaison_claims_institutions_sansnom_sw_corr.json",dico_slmt_insti)

2592


In [15]:
dico_sans_insti = {}
for chemin in dico_strengths.keys():
    if chemin not in dico_institutions :
        dico_sans_insti[chemin]=dico_strengths[chemin]

print(len(dico_sans_insti))
#print(dico_sans_insti)
ecrire_json("comparaison_claims_noinstitutions_sansnom_sw_corr.json",dico_sans_insti)

5281


In [12]:
#voir quelles institutions sont présentes (liste des valeurs du dico sans doublons avec leur nb d'occu)
dico_institutions = ouvrir_json("dico_institutions1000_corr.json")
nb_insti = {}
for liste_insti in dico_institutions.values():
    for insti in liste_insti:
        if insti not in nb_insti:
            nb_insti[insti]=0
        nb_insti[insti]+=1

nb_insti = sorted(nb_insti.items(), key=lambda x:x[1],reverse=True)
nb_insti_trie = dict(nb_insti)
print(nb_insti_trie)
ecrire_json("dico_nb_insti_sw_corr.json",nb_insti_trie)
#vérifier pour les GAFAM si bien des affiliations

{'Microsoft': 344, 'University of California': 320, 'University of Edinburgh': 222, 'Google': 189, 'University of Pennsylvania': 156, 'Stanford University': 151, 'Peking University': 140, 'Johns Hopkins University': 136, 'University of Washington': 122, 'Tsinghua University': 120, 'University of Cambridge': 116, 'Columbia University': 116, 'MIT': 102, 'Facebook': 81, 'University of Tokyo': 81, 'Cornell University': 69, 'New York University': 69, 'Amazon': 63, 'Massachusetts Institute of Technology': 57, 'University of Texas at Austin': 51, 'University of Copenhagen': 47, 'University of Melbourne': 47, 'University of Michigan': 47, 'University of Toronto': 47, 'Kyoto University': 39, 'Harvard University': 37, 'University of Manchester': 27, 'University of Oxford': 25, 'University of Chicago': 20, 'Northwestern University': 18, 'University College London': 18, 'Duke University': 16, 'University of Minnesota': 15, 'Yale University': 14, 'Washington University': 13, 'Imperial College Londo